In [2]:
import logging ;logger = logging.getLogger(__name__); logging.basicConfig(level='INFO'); load('main.sage')

skmod = QuantumAPolynomial('4_1')
skmod.compute_skein_module()

new_basis = [skmod.gens_dict['qrt2'],skmod.meridian,skmod.longitude]

for vec in skmod.invariant_sublattice.basis():
    if vec not in span(new_basis):
        new_basis.append(vec)

inv_lattice = skmod.invariant_sublattice.submodule_with_basis(new_basis)

restricted_omega = inv_lattice.basis_matrix()*skmod.omega_with_q*(inv_lattice.basis_matrix().transpose())


INFO:__main__:A-polynomial for 4_1:	M^8*L - M^6*L + M^4*L^2 - 2*M^4*L + M^4 - M^2*L + L
INFO:__main__:After changing L to -L our A-polynomial is divisible by the reference one!


Defining qrt2, M, L, w0, w0i


In [ ]:
L.parent()

In [ ]:
inv_lattice.dimension()


# Set up g_algebra

In [3]:
S = FractionField(QQ['q'])
q = S.gen()
#A.<x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36> = FreeAlgebra(S,36)

In [4]:
dim_wo_q = inv_lattice.dimension()-1
A2 = FreeAlgebra(S,2*dim_wo_q,['M','L'] + ['x{}'.format(i) for i in range(1,dim_wo_q-2+1)]+['Mi','Li']+['xi{}'.format(i) for i in range(1,dim_wo_q-2+1)])
FA_gens_dict = {A2.gen(i) : [0]*(i) + [1] + [0]*(2*dim_wo_q-i-1) for i in range(A2.ngens())}


In [5]:
doubled_res_omega = Matrix.block([[restricted_omega[1:,1:],-restricted_omega[1:,1:]],[-restricted_omega[1:,1:],restricted_omega[1:,1:]]])

In [6]:
import itertools
relations_FA = {}
for g2,g1 in itertools.combinations(FA_gens_dict.keys(),2):
    #print(matrix(x_gens_dict[g1])*(restricted_omega*(matrix(x_gens_dict[g2]).transpose())))
    pow = (matrix(FA_gens_dict[g1])*doubled_res_omega*(matrix(FA_gens_dict[g2]).transpose()))[0][0]
    if pow != 0:
        this_rel = [g1*g2,q**int(pow)*g2*g1]
        #this_rel.sort(reverse=True)
        relations_FA[this_rel[0]] = this_rel[1]
        
    
H2 = A2.g_algebra(relations_FA)

In [7]:
H2.inject_variables()
H2
H2(A2('x1*xi1'))-1

Defining M, L, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20, x21, x22, x23, x24, x25, x26, x27, x28, x29, x30, x31, x32, x33, x34, Mi, Li, xi1, xi2, xi3, xi4, xi5, xi6, xi7, xi8, xi9, xi10, xi11, xi12, xi13, xi14, xi15, xi16, xi17, xi18, xi19, xi20, xi21, xi22, xi23, xi24, xi25, xi26, xi27, xi28, xi29, xi30, xi31, xi32, xi33, xi34


x1*xi1 - 1

In [8]:
# little check: the commutation relation agrees on M and L:
print(matrix(skmod.longitude)*skmod.omega_with_q*(matrix(skmod.meridian).transpose()))
print(L*M)

[-1]
1/q*M*L


## old g_algebra setup - without inverses

In [ ]:
x_gens = [v for k,v in A.gens_dict().items() if k[0] == 'x']
x_gens_dict = {x_gens[i]: [0]*(i+1) + [1] + [0]*(37-i-2) for i in range(len(x_gens))}

In [ ]:
import itertools
relations = {}
for g2,g1 in itertools.combinations(x_gens,2):
    #print(matrix(x_gens_dict[g1])*(restricted_omega*(matrix(x_gens_dict[g2]).transpose())))
    pow = (matrix(x_gens_dict[g1])*restricted_omega*(matrix(x_gens_dict[g2]).transpose()))[0][0]
    if pow != 0:
        this_rel = [g1*g2,q**int(pow)*g2*g1]
        #this_rel.sort(reverse=True)
        relations[this_rel[0]] = this_rel[1]
        
    
H = A.g_algebra(relations)



In [ ]:
H.inject_variables()

In [ ]:
relations

# Get relations into g_algebra

In [9]:
def get_doubled_lattice_coord(coord):
    """
    Assumes the first coordinate is q, which can be negative and shouldn't be doubled
    ex:
    (1,-1,2) ->  ( 1,0,2,1,0)
    (-1,-1,2) -> (-1,0,2,1,0)
    """
    
    pos = vector([max(coord[i],0) for i in range(len(coord))])
    neg = vector([min(coord[i],0) for i in range(len(coord))])
    return list(pos) + list(-1*neg[1:])

get_doubled_lattice_coord(skmod.monomial_relations[1])

def get_g_alg_element_from_coord(coord,g_alg):
    """
    Assumes the coordinate is for the skein algebra, i.e. has q as the first coordinate and needs to be doubled
    """
    doubled_coord = get_doubled_lattice_coord(coord)
    #logger.debug("coord: {}".format(coord))
    #logger.debug("doubled_coord: {}".format(doubled_coord))
    #logger.debug("ngens: {0}, double_coord length:{1}".format(g_alg.ngens(),len(doubled_coord)))
    qq = g_alg.base().gen()
    free_alg = g_alg.free_algebra()
    # make the monomial as a string:
    without_q = '*'.join([
        '{0}^({1})'.format(g_alg.gen(i),doubled_coord[i])
        for i in range(1,len(doubled_coord)-1) if doubled_coord[i] != 0
    ])
    return qq^doubled_coord[0]*g_alg(free_alg(without_q))

get_g_alg_element_from_coord(inv_lattice.basis_matrix()*skmod.monomial_relations[1],H2)

q*L^2*x1^5*x7^3*x9^4*x10*x14^3*x15*x19^5*x20^4*x29^2*x30^4*x33^6*xi6^3*xi8^5*xi21^2

In [10]:
monomial_relations = [get_g_alg_element_from_coord(inv_lattice.basis_matrix()*c,H2)-1 for c in skmod.monomial_relations]
crossing_rel = [
    reduce(H2.summation,
           [-1]+[
               get_g_alg_element_from_coord(inv_lattice.basis_matrix()*mon,H2)
               for mon in relation]
          ) for relation in skmod.crossing_relations]
inverses_relations = [M*Mi-1,L*Li-1]+[H2(A2('x{0}*xi{1}'.format(j,j)))-1 for j in range(1,dim_wo_q-2+1)]

relations_ideal = H2.ideal(monomial_relations + crossing_rel+inverses_relations)

In [13]:
# This ideal should be generated by the quantum A-polynomial
# it is infact (1).
# The whole relations ideal isn't (1).
# If I include either the monomial relations or the crossing relations I get (1).
relations_ideal.elimination_ideal([g for name,g in H2.gens_dict().items() if name[0]=='x'])

Left Ideal (1) of Noncommutative Multivariate Polynomial Ring in M, L, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20, x21, x22, x23, x24, x25, x26, x27, x28, x29, x30, x31, x32, x33, x34, Mi, Li, xi1, xi2, xi3, xi4, xi5, xi6, xi7, xi8, xi9, xi10, xi11, xi12, xi13, xi14, xi15, xi16, xi17, xi18, xi19, xi20, xi21, xi22, xi23, xi24, xi25, xi26, xi27, xi28, xi29, xi30, xi31, xi32, xi33, xi34 over Fraction Field of Univariate Polynomial Ring in q over Rational Field, nc-relations: {L*M: 1/q*M*L,
 x5*M: q*M*x5,
 x6*M: 1/q*M*x6,
 x13*M: 1/q*M*x13,
 x19*M: 1/q*M*x19,
 x20*M: q*M*x20,
 x29*M: 1/q*M*x29,
 Li*M: q*M*Li,
 xi5*M: 1/q*M*xi5,
 xi6*M: q*M*xi6,
 xi13*M: q*M*xi13,
 xi19*M: q*M*xi19,
 xi20*M: 1/q*M*xi20,
 xi29*M: q*M*xi29,
 x1*L: 1/(q^3)*L*x1,
 x2*L: (q^2)*L*x2,
 x3*L: q*L*x3,
 x4*L: q*L*x4,
 x5*L: q*L*x5,
 x6*L: 1/(q^2)*L*x6,
 x7*L: 1/q*L*x7,
 x8*L: q*L*x8,
 x10*L: 1/q*L*x10,
 x11*L: q*L*x11,
 x13*L: 1/q*L*x13,
 x14*L: 1/q*L*x14,
 x15*L: q*L*x1

## Commutative case

In [14]:
H_comm = A2.g_algebra({})

comm_monomial_relations = [get_g_alg_element_from_coord(inv_lattice.basis_matrix()*c,H_comm)-1 for c in skmod.monomial_relations]
comm_crossing_rel = [
    reduce(H_comm.summation,
           [-1]+[
               get_g_alg_element_from_coord(inv_lattice.basis_matrix()*mon,H_comm)
               for mon in relation]
          ) for relation in skmod.crossing_relations]
comm_inverses_relations = [H_comm(A2('M*Mi'))-1,H_comm(A2('L*Li'))-1]+[H_comm(A2('x{0}*xi{1}'.format(j,j)))-1 for j in range(1,dim_wo_q-2+1)]

comm_relations_ideal = H_comm.ideal(comm_monomial_relations + comm_crossing_rel + comm_inverses_relations)

In [15]:
all_comm_x_variables = [g for name,g in H_comm.gens_dict().items() if name[0]=='x']

In [ ]:
comm_relations_ideal.elimination_ideal(all_comm_x_variables)

# David Code

In [ ]:
MLspan = t41.invariant_sublattice.span([t41.meridian,t41.longitude])
dir(t41.invariant_sublattice)
B=[t41.gens_dict['qrt2'],t41.meridian,t41.longitude]
for b in t41.invariant_sublattice.basis():
    if not b in span(B):
        B.append(b)

new_invariant_sublattice=t41.invariant_sublattice.submodule_with_basis(B)
B.remove(t41.gens_dict['qrt2'])

def vector_to_monomial(v):
    assert v in t41.invariant_sublattice
    gens_list=[]
    for i in range(len(v)):
        if v[i]>0:
            gens_list.append(QuantumTorus.gen(i)^v[i])
        if v[i]<0:
            gens_list.append(QuantumTorus.gen(i+len(B))^-v[i])
    return mul(gens_list)

R.<A>=PolynomialRing(QQ)
F=R.fraction_field()
names = ["M","L"] + ['x{}'.format(i) for i in range(len(B)-2)] + ["Mi","Li"] + ['x{}i'.format(i) for i in range(len(B)-2)]
QuantumTorusFA = FreeAlgebra(F,len(names),names)
QuantumTorusFA.inject_variables()
crossrels = {QuantumTorusFA(names[i])*QuantumTorusFA(names[j]): A^((matrix(B[i])*t41.omega_with_q*matrix(B[j]).transpose())[0,0])*QuantumTorusFA(names[j])*QuantumTorusFA(names[i]) for i in range(len(B)) for j in range(len(B)) if i>j} | {QuantumTorusFA(names[i+len(B)])*QuantumTorusFA(names[j+len(B)]): A^((matrix(B[i])*t41.omega_with_q*matrix(B[j]).transpose())[0,0])*QuantumTorusFA(names[j+len(B)])*QuantumTorusFA(names[i+len(B)]) for i in range(len(B)) for j in range(len(B)) if i>j} | {QuantumTorusFA(names[i+len(B)])*QuantumTorusFA(names[j]): A^((matrix(B[j])*t41.omega_with_q*matrix(B[i]).transpose())[0,0])*QuantumTorusFA(names[j])*QuantumTorusFA(names[i+len(B)]) for i in range(len(B)) for j in range(len(B))} 
QuantumTorus=QuantumTorusFA.g_algebra(crossrels)